In [22]:
# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.metrics import BinaryAccuracy, AUC
# Function to create model, required for KerasClassifier

## DL Model Creation

In [23]:
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_shape=(8,), activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

## Data Preprocessing

In [24]:
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)
# load dataset
dataset = np.loadtxt("data.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:, 0:8]
Y = dataset[:, 8]

## Model Instatiation

In [34]:
from keras import backend as K


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [36]:
# create model
model = KerasClassifier(model=create_model, verbose=0, metrics=['acc', f1_m, precision_m, recall_m], metrics__threshold=0.65)


### Results without using Cross Validated Hyperparameter Tuning

In [41]:
result = model.fit(X, Y, validation_split=0.3, epochs=10, verbose=0, batch_size=40)

In [46]:
predict = model.score(X,Y)
print(predict)

0.5104166666666666


## Hyperparameter tuning using GridSearchCV

In [9]:
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

In [48]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

f"The final score is: {grid_result.score(X,Y)}"

Best: 0.691406 using {'batch_size': 20, 'epochs': 100}
0.619792 (0.041504) with: {'batch_size': 10, 'epochs': 10}
0.667969 (0.019401) with: {'batch_size': 10, 'epochs': 50}
0.688802 (0.035132) with: {'batch_size': 10, 'epochs': 100}
0.516927 (0.051953) with: {'batch_size': 20, 'epochs': 10}
0.647135 (0.017566) with: {'batch_size': 20, 'epochs': 50}
0.691406 (0.011049) with: {'batch_size': 20, 'epochs': 100}
0.597656 (0.020915) with: {'batch_size': 40, 'epochs': 10}
0.634115 (0.023939) with: {'batch_size': 40, 'epochs': 50}
0.654948 (0.015733) with: {'batch_size': 40, 'epochs': 100}
0.515625 (0.068998) with: {'batch_size': 60, 'epochs': 10}
0.664062 (0.003189) with: {'batch_size': 60, 'epochs': 50}
0.630208 (0.009744) with: {'batch_size': 60, 'epochs': 100}
0.638021 (0.011201) with: {'batch_size': 80, 'epochs': 10}
0.605469 (0.024910) with: {'batch_size': 80, 'epochs': 50}
0.626302 (0.019225) with: {'batch_size': 80, 'epochs': 100}
0.630208 (0.031466) with: {'batch_size': 100, 'epochs':

'The final score is: 0.7421875'